# Comparisons using the BatchStudy class

In this notebook, we will be going through the `BatchStudy` class and will be discussing how different models, experiments, chemistries, etc. can be compared with each other using the same.

## Comparing models
We start by creating a simple script to compare `SPM`, `SPMe` and `DFN` model with the default parameters.

In [ ]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm

# loading up 3 models to compare
dfn = pybamm.lithium_ion.DFN()
spm = pybamm.lithium_ion.SPM()
spme = pybamm.lithium_ion.SPMe()

The `BatchStudy` class requires a dictionary of models, and all the default values for a given model are used if no additional parameter is passed in.

In [ ]:
models = {
    "dfn": dfn,
    "spm": spm,
    "spme": spme,
}

# creating a BatchStudy object
batch_study = pybamm.BatchStudy(models=models)

# solving and plotting the comparison
batch_study.solve(t_eval=[0, 3600])
batch_study.plot()

`BatchStudy` by default requires equal number of items in all the dictionaries passed, which can be changed by setting the value of `permutations` to `True`. When set `True`, a cartesian product of all the available items is taken.

For example, here we pass 3 models but only 1 parameter value, hence it is necessary to set `permutations` to `True`. Here, the given parameter value is used for all the provided models.

In [ ]:
# passing parameter_values as a dictionary
parameter_values = {"Chen2020": pybamm.ParameterValues("Chen2020")}

# creating a BatchStudy object and solving the simulation
batch_study = pybamm.BatchStudy(
    models=models, parameter_values=parameter_values, permutations=True
)
batch_study.solve(t_eval=[0, 3600])
batch_study.plot()

## Comparing parameters

`BatchStudy` can also be used to compare different things (like affect of changing a parameter's value) on a single model.

In the following cell, we compare different values of `"Curent function [A]"` using the `Single Paritcle Model with electrolyte`.


In [ ]:
model = {"spme": spme}

# populating a dictionary with 3 same parameter values
parameter_values = {
    "Chen2020_1": pybamm.ParameterValues("Chen2020"),
    "Chen2020_2": pybamm.ParameterValues("Chen2020"),
    "Chen2020_3": pybamm.ParameterValues("Chen2020"),
}

# different values for "Current function [A]"
current_values = [4.5, 4.75, 5]

# changing the value of "Current function [A]" in all the parameter values present in the
# parameter_values dictionary
for _, v, current_value in zip(
    parameter_values.keys(), parameter_values.values(), current_values
):
    v["Current function [A]"] = current_value

# creating a BatchStudy object with permutations set to True to create a cartesian product
batch_study = pybamm.BatchStudy(
    models=model, parameter_values=parameter_values, permutations=True
)
batch_study.solve(t_eval=[0, 3600])

# generating the required labels and plotting
labels = [f"Current function [A]: {current}" for current in current_values]
batch_study.plot(labels=labels)

`BatchStudy` also includes a `create_gif` method which can be used to create a GIF of the simulation.

In [ ]:
# using less number of images in the example
# for a smoother GIF use more images
batch_study.create_gif(number_of_images=5, duration=0.2, output_filename="batch.gif")

Displaying the GIF using markdown-

![plot](https://user-images.githubusercontent.com/74055102/142717896-8152e816-71b1-47a7-b557-57cf6dbfc839.gif)

## Using experiments

Experiments can also be specified for comparisons, and they are also passed as a dictionary (a dictionary of `pybamm.Experiment`) in the `BatchStudy` class.

In the next cell, we compare a single experiment, with a single model, but with a varied parameter value.


In [ ]:
pybamm.set_logging_level("NOTICE")

# using the cccv experiment with 10 cycles
cccv = pybamm.Experiment(
    [
        (
            "Discharge at C/10 for 10 hours or until 3.3 V",
            "Rest for 1 hour",
            "Charge at 1 A until 4.1 V",
            "Hold at 4.1 V until 50 mA",
            "Rest for 1 hour",
        )
    ]
    * 10,
)

# creating the experiment dict
experiment = {"cccv": cccv}

# populating a dictionary with 3 same parameter values (Mohtat2020 chemistry)
parameter_values = {
    "Mohtat2020_1": pybamm.ParameterValues("Mohtat2020"),
    "Mohtat2020_2": pybamm.ParameterValues("Mohtat2020"),
    "Mohtat2020_3": pybamm.ParameterValues("Mohtat2020"),
}

# different values for the parameter "SEI open-circuit potential [V]"
sei_oc_v_values = [2.0e-4, 2.7e-4, 3.4e-4]

# updating the value of "SEI open-circuit potential [V]" in all the dictionary items
for _, v, sei_oc_v in zip(
    parameter_values.keys(), parameter_values.values(), sei_oc_v_values
):
    v.update(
        {"SEI open-circuit potential [V]": sei_oc_v},
    )

# creating a Single Particle Model with "electron-mitigation limited" SEI
model = {"spm": pybamm.lithium_ion.SPM({"SEI": "electron-migration limited"})}

# creating a BatchStudy object with the given experimen, model and parameter_values
batch_study = pybamm.BatchStudy(
    models=model,
    experiments=experiment,
    parameter_values=parameter_values,
    permutations=True,
)

# solving and plotting the result
batch_study.solve(initial_soc=1)

labels = [
    f"SEI open-circuit potential [V]: {inner_sei_oc_v}"
    for inner_sei_oc_v in sei_oc_v_values
]
batch_study.plot(labels=labels)

The difference in the individual plots is not very well visible in the above slider plot, but we can access all the simulations created by `BatchStudy` (`batch_study.sims`) and pass it to `pybamm.plot_summary_variables` to plot the summary variables (more details on "summary variables" are available in the [`simulating-long-experiments`](./simulations_and_experiments/simulating-long-experiments.ipynb) notebook).

## Comparing summary variables

In [ ]:
pybamm.plot_summary_variables([sim.solution for sim in batch_study.sims])

Other than the above examples, the `BatchStudy` class can be used to compare a lot of different configurations, like models with different SEIs or a model with reversible and irreversible lithium plating, etc.

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()